In [1]:
import json
with open('transfer_results.json', 'r') as fp:
    tx_data = json.load(fp)

In [2]:
import pandas as pd
df = pd.DataFrame(tx_data)
df['mint'] = df['from'] == '0x0000000000000000000000000000000000000000'
df.columns

Index(['from', 'to', 'value', 'transferIndex', 'borrowCap', 'supplyCap',
       'totalVariableDebt', 'variableBorrowRate', 'blockNumber',
       'blockTimestamp', 'transactionHash', 'mint'],
      dtype='object')

In [3]:
minters = df[df['mint']]['to']
len(minters)

7646

## First order flows

We do volume, so we don't avoid double counting here (unlike w/ num of txs)

### Hoarders

In [33]:
senders = set(df['from'])
hoarders = list(set([user for user in minters if user not in senders]))
print("Total hoarders:", len(hoarders))
print("Total amount:", sum(df[df['to'].isin(hoarders)  & (df['from'] == '0x0000000000000000000000000000000000000000')]['value'].astype(float)) * 1e-18)

Total hoarders: 141
Total amount: 4989.569460977369


In [21]:
pd.set_option('display.max_colwidth', None)

In [22]:
df[df['to'].isin(hoarders)  & (df['from'] == '0x0000000000000000000000000000000000000000')]

,from,to,value,transferIndex,borrowCap,supplyCap,totalVariableDebt,variableBorrowRate,blockNumber,blockTimestamp,transactionHash,mint
12,0x0000000000000000000000000000000000000000,0x9677e256755e1718e7e455b3bf9bce84c6db6b2e,1000000000000000000,12,0,0,86899927822861219329019,15000000000000000000000000,17699345,1689430967,0x0b33d05bb81f91de8bf67f6ed91157826e4fa348bf75f012e03974e89c91697d,True
15,0x0000000000000000000000000000000000000000,0x7013674db1086b85e31a847ecbeb3f06bac373df,60000000000000000000,15,0,0,251960942250990255446694,15000000000000000000000000,17699356,1689431111,0x82d5bd10867180e9efe183781731c8e37afe1e65495259aa3f6f5388a1231837,True
25,0x0000000000000000000000000000000000000000,0xdda1e0b2c225fc5de845655acad8bad5cb294d81,30000000000000000000,25,0,0,277110957918852695782556,15000000000000000000000000,17699366,1689431231,0x99824c62273f729f834b63558266a8837783d239739ffd86184393ddb25345ed,True
58,0x0000000000000000000000000000000000000000,0xfe9d3b6f60f5ca2c76b4b682e497cf04e26e7257,50000000000000000000,58,0,0,2221633886029215806475573,15000000000000000000000000,17699509,1689433007,0x1b0e186150de262feddf303507590b434a14ce8179b42dc8964094cb8cd3c432,True
186,0x0000000000000000000000000000000000000000,0x26199b5dd7637d8ddd76fcfe31ec1eadee25e275,50000000000000000000,186,0,0,2411086519505632602906348,15000000000000000000000000,17700287,1689442547,0x1ff194bc22d7fc25b1785172136b4c58c81ae9bbb58fb2a2131ee7f6edf5501d,True
...,...,...,...,...,...,...,...,...,...,...,...,...
79716,0x0000000000000000000000000000000000000000,0xb32709456b3a6a31b8acfd716ef9a28b9b4117a8,5000000000000000000,79716,116000000,0,107008314858150574125447027,60000000000000000000000000,20649797,1725122339,0x3a1e9342658a3bf2a4d702d4104ad3061d295e966612811c3e55e710123ead23,True
80089,0x0000000000000000000000000000000000000000,0x5a6a19033e3790891a00d97be1931e45b2a78ff7,24652100000000,80089,116000000,0,107571390386639323512831726,60000000000000000000000000,20654742,1725181883,0x134bb8c0bda6d531653ec3ee7c626b43df21f1c17de8ec64377220fd56a9261b,True
80128,0x0000000000000000000000000000000000000000,0xc41a6568889fd2017650efcb959653a55a46018b,50000000000000000,80128,116000000,0,107572503641510393537883744,60000000000000000000000000,20655344,1725189155,0x78034559290ce48d9f54018b354cc7d0081cb3d2784ef4441c1578569f4b2717,True
81095,0x0000000000000000000000000000000000000000,0x9055631a4843a4728f82daee87b64f884f23473c,391799494800000000,81095,125000000,0,108740331375063687319778116,60000000000000000000000000,20669446,1725359207,0xfb7e39a9af7d68ca5adb6d1edfb41a5b8034250cb25152fcbd2b6581f5968dde,True


### Contracts

In [5]:
minter_txs = df[df['from'].isin(minters)].copy()
#minter_txs = minter_txs.drop_duplicates(['from', 'to'])

In [6]:
minter_txs['value'] = minter_txs['value'].astype(float) * 1e-18

In [7]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# We are going to cut the dataframe at March 15th (start of Merit program)

In [10]:
minter_txs['blockTimestamp'] = minter_txs['blockTimestamp'].astype(int)
minter_txs_before = minter_txs[minter_txs['blockTimestamp'] <= 1710421200].copy()
minter_txs_after = minter_txs[minter_txs['blockTimestamp'] > 1710421200].copy()

In [185]:
valuables = minter_txs[['to', 'value']].groupby('to').sum().sort_values('value', ascending=False)
valuables[:20]

,value
to,
0x00907f9921424583e7ffbfedf84f92b7b2be4977,111223491.79
0x1a88df1cfe15af22b3c4c783d4e6f7f9e0c1885d,62748484.39
0x9008d19f58aabd9ed0d60971565aa8510560ab41,50775019.65
0xdef171fe48cf0115b1d80b88dc8eab59176fee57,39857643.78
0xba12222222228d8ba445958a75a0704d566bf2c8,27570367.13
0xb639d208bcf0589d54fac24e655c79ec529762b8,26732582.60
0x5c95d4b1c3321cf898d25949f41d50be2db5bc1d,23499266.43
0xe37e799d5077682fa0a244d46e5649f71457bd09,19991553.03
0x0000000000000000000000000000000000000000,7501864.43


In [4]:
import os
import requests

API_KEY = os.environ.get('ETHERSCAN_TOKEN')
contract_names = {'0x00907f9921424583e7ffbfedf84f92b7b2be4977': 'GhoAToken',
 '0x5c95d4b1c3321cf898d25949f41d50be2db5bc1d': 'UniswapV3Pool',
 '0x1a88df1cfe15af22b3c4c783d4e6f7f9e0c1885d': 'GHO Staking',
 '0x9008d19f58aabd9ed0d60971565aa8510560ab41': 'CowSwap',
 '0xdef171fe48cf0115b1d80b88dc8eab59176fee57': 'ParaSwap',
 '0x383e7acd889bf57b0d79a584009cb570534ab518': 'UniswapV3Pool',
 '0xe37e799d5077682fa0a244d46e5649f71457bd09': '1inch',
 '0x54eebc36527fe2e5624051e3c895810d7b68bcfc': 'UniswapV3Pool',
 '0xba12222222228d8ba445958a75a0704d566bf2c8': 'Balancer',
 '0x74de5d4fcbf63e00296fd95d33236b9794016631': 'AirSwap',
 '0xad3b67bca8935cb510c8d18bd45f0b94f54a968f': '1inch',
 '0x8d8404f8cca4c8834ca3cab1e54887ae47724bee': '1inch',
 '0x5756880b6a1eaba0175227bf02a7e87c1e02b28c': 'GHO CCIP',
 '0x22f9dcf4647084d6c31b2765f6910cd85c178c18': '0x Exchange',
 '0xbbec426df87537ad0bec0e899007be441427d0f8': 'UniswapV3Pool',
 '0xf0d4c12a5768d806021f80a262b4d39d26c58b8d': 'Curve',
 '0x3208684f96458c540eb08f6f01b9e9afb2b7d4f0': '1inch',
 '0x92f3f71cef740ed5784874b8c70ff87ecdf33588': '1inch',
 '0xf081470f5c6fbccf48cc4e5b82dd926409dcdd67': 'Kyber Network',
 '0x6f42b83ecda76a8313fd8a45ca18a3fdfd37bbc7': 'Gearbox',
 '0x1231deb6f5749ef6ce6943a275a1d3e7486f4eae': 'Li.Fi',
 '0x14cf6d2fe3e1b326114b07d22a6f6bb59e346c67': 'Maverick',
 '0x16c6521dff6bab339122a0fe25a9116693265353': 'Curve',
 '0xf4d34d7fdd2ed5d57c6f583237bb038fc3839d17': '1inch',
 '0x9a83b11e50ff5cc544f1401f35807b7c07de791d': 'UniswapV3Pool',
 '0x28104d4f703ee5b5011cefe106f54efd56f33f95': 'Odos',
 '0x5f0000d4780a00d2dce0a00004000800cb0e5041': 'ParaSwap',
 '0x670a72e6d22b0956c0d2573288f82dcc5d6e3a61': 'Curve',
 '0xbb33ed8c1148e62411a97d25390fd48698fd937e': '1inch',
 '0xd8533305aa8d9eb5681f15f1461e02c1601eed72': 'UniswapV3Pool',
 '0x5f515f6c524b18ca30f7783fb58dd4be2e9904ec': '1inch',
 '0x050ebe3dbb4b3a3526735b04cc3d96c80609ee7e': 'Maverick',
 '0x86152df0a0e321afb3b0b9c4deb813184f365ada': 'Curve',
 '0xbf683b7aaf9accea6bff8e926e03f6ceb98b525a': '1inch',
 '0xd1742b3c4fbb096990c8950fa635aec75b30781a': '1inch',
 '0x74345504eaea3d9408fc69ae7eb2d14095643c5b': 'Curve',
 '0xc8c9cfad493cffffd6684da0f45d141e33b6ef27': '1inch',
 '0x0d67b27c340887da87344c344c20ce60a8eb84c5': 'Maverick',
 '0x99a58482bd75cbab83b27ec03ca68ff489b5788f': 'Curve',
 '0xa8376f53391a041c8236a232f7f019ea76eed86d': 'Odos',
 '0x1a22c1886196101babcac1bf4223294ab04c1d66': 'Odos',
 '0xc48c6d400de27081df85d55100c7b06f190c764a': 'Maverick',
 '0x9f274d28fedce05153f9c810b8070277eaf8c031': 'Maverick',
 '0x635ef0056a597d13863b73825cca297236578595': 'Curve',
 '0x0b8a49d816cc709b6eadb09498030ae3416b66dc': '1inch',
 '0x1d7405df25fd2fe80390da3a696dcfd5120ca9ce': 'ParaSwap',
 '0x8dd418fa80b7f37856e36199e67013f00e5d7a61': 'Maverick',
 '0xd08d0006f00040b400180f9500b00c5026ac0900': 'ParaSwap',
 '0x19ea2e6f21bdfc894abf09fa179d59f6c0e0797b': '1inch',
 '0x63e988f4b30245f9f4ee898531e0be3fee20b170': '1inch',
                 #########
  '0x686f8d21520f4ecec7ba577be08354f4d1eb8262': 'GHO Stability Module',
                  '0x0d8effc11df3f229aa1ea0509bc9dfa632a13578': 'GHO Stability Module',
                  '0x0cbc13b84286007ec4f17cd0f9dc7d7ca6ee4fff': 'Odos',
                  '0x7d2b63a9ab475397d9c247468803f25cf6523b76': 'Odos',
                  '0xc351e45db65d68585e180795537563d33b3716e7': 'ParaSwap',
                  '0x9c211ec76380bae8e370a517df95075b40d34cdf': 'Maverick',
                  '0x005c78a48b482c1733c7cf958e65d90d3d40554b': 'ParaSwap',
                  '0x9c594c2e2e2e5aa300be12596215188c324c3e7c': 'Pendle',
                  '0x3d20601ac0ba9cae4564ddf7870825c505b69f1a': 'Pendle',
                  '0xe4e9278aed2482248979768a71ca4bfd0c999988': "MultiSigs",
                  '0xe705b1d26b85c9f9f91a3690079d336295f14f08': "MultiSigs",
                  '0x205e795336610f5131be52f09218af19f0f3ec60': "MultiSigs",
                  "0xd211a02a0adde56bb7f9700f49d4ba832adc7ddf": "MultiSigs",
                  "0xe8d4d93d9728bd673b0197673a230f62255c7846": "MultiSigs",
                  '0x6e7058c91f85e0f6db4fc9da2ca41241f5e4263f': "Notional",
                  '0xff98eb65eb5a727d4354d37abb3ef27537a56b60': "DeFi Saver",
                  '0xc2b92b6c69e5c3b6b29385c1a17fee906e0ca910': "DeFi Saver",
                  '0x8c82d963eb282cb7a751e551e3997ba66c2c8237': "DeFi Saver",
                  "0xb639d208bcf0589d54fac24e655c79ec529762b8": "DeFi Saver",
                  "0x0e18476415d63882f5a82ad0ecae0072e42ce992": "DeFi Saver",
                  "0xbb67b81dd080a406227a38965d0393f396ddecbc": "DeFi Saver",
                  "0x02dcc3438bb7e46f6bf671c72e98d503c0bce520": "DeFi Saver",
                  "0x3f64bc83f10c4f11a2c7fbd9190949b1c8d54a43": "DeFi Saver",
                  "0xe27baebd7b14602de3797974db9f5f4f8dcb6679": "MEV Bots",
                  "0xa87f4f8b87abf5237c16971e406eef0e2d541c9e": "MEV Bots",
                  "0xd5e74de4385ef0eb1bb4db05a6a504f04d92e79d": "MEV Bots",
                  "0xe20cd9377c204a27952f8b41075f0b8bd1ceec3d": "MEV Bots",
                  "0xf1d2d880c5dde7cc912636c2b4d080b3fe5f7b50": "MEV Bots",
                  "0x00000000009e50a7ddb7a7b0e2ee6604fd120e49": "MEV Bots",
                  "0x2593154f192f46f65a4a0315a5d77556d5d3d264": "MEV Bots",
                  "0x0000000000000000000000000000000000000000": "Burn",
                  
                 }

In [41]:
for contract in valuables[:30].index:
    if contract in contract_names: 
        continue
    URL = f"https://api.etherscan.io/api?module=contract&action=getsourcecode&address={contract}&apikey={API_KEY}"
    r = requests.get(URL)
    if r.json()['result'][0]['Implementation']:
        impl_contract = r.json()['result'][0]['Implementation']
        URL = f"https://api.etherscan.io/api?module=contract&action=getsourcecode&address={impl_contract}&apikey={API_KEY}"
        r = requests.get(URL)        
    contract_names[contract] = (r.json()['result'][0]['ContractName'])
    print(contract, (r.json()['result'][0]['ContractName']))

In [188]:
for contract in valuables[:60].index:
    if contract in contract_names: 
        continue
    URL = f"https://api.etherscan.io/api?module=contract&action=getsourcecode&address={contract}&apikey={API_KEY}"
    r = requests.get(URL)
    if r.json()['result'][0]['Implementation']:
        impl_contract = r.json()['result'][0]['Implementation']
        URL = f"https://api.etherscan.io/api?module=contract&action=getsourcecode&address={impl_contract}&apikey={API_KEY}"
        r = requests.get(URL)        
    #contract_names[contract] = (r.json()['result'][0]['ContractName'])
    print(contract, (r.json()['result'][0]['ContractName']))

0x0000000000000000000000000000000000000000 
0x9c6a52c6f736ba3cd9c21bd13eebe08760f931e5 
0x2079c29be9c8095042edb95f293b5b510203d6ce 
0x78d492949664cc4c2dd3db1448cc9b9b5e70c45a 
0x2f1e7ced9f5f7edc39459c28f5d7eb4823c60030 
0x00000000009e50a7ddb7a7b0e2ee6604fd120e49 
0x2593154f192f46f65a4a0315a5d77556d5d3d264 
0x619ad2d02dbee6eba3cdbda3f98430410e892882 InstaFlashAggregator
0xd2d53f63173a5505f8539dfcec88769ef65d47c2 Wrapper


In [108]:
valuables[(~valuables.index.isin(contract_names)) & (valuables.index.isin(not_eoas))][:10]

,value
to,
0x619ad2d02dbee6eba3cdbda3f98430410e892882,374310.06
0xdb2a2bba2346e79352f5eb612613437bcae8d80f,250000.00
0xd2d53f63173a5505f8539dfcec88769ef65d47c2,217704.89
0x12e55236d0743999716ebacbd1fe07f63719b0df,200000.00
0xe08d97e151473a848c3d9ca3f323cb720472d015,197618.17
0x93ebc3ca85f96afd72edb914e833fe18888de179,154609.68
0xa70c8401c058b6198e1cb085091de13498cec0dc,151023.00
0x7e15eb462cdc67cf92af1f7102465a8f8c784874,149439.38
0x64545160d28fd0e309277c02d6d73b3923cc4bfa,135325.30


In [190]:
minter_txs.copy()['to'].apply(lambda x: contract_names.get(x, "Others")).value_counts()

GhoAToken               3054
UniswapV3Pool           2574
ParaSwap                1514
GHO Staking             1317
CowSwap                 1229
Others                  1100
1inch                    922
DeFi Saver               852
Curve                    289
Balancer                 260
AirSwap                  212
Maverick                 189
MEV Bots                  81
0x Exchange               79
GHO CCIP                  70
Odos                      64
Li.Fi                     62
Kyber Network             58
Gearbox                   56
Burn                      42
Pendle                    26
Notional                  21
MultiSigs                 18
GHO Stability Module      15
Name: to, dtype: int64

In [12]:
minter_txs_after.copy()['to'].apply(lambda x: contract_names.get(x, "Others")).value_counts()

GhoAToken               1511
GHO Staking             1185
ParaSwap                 720
CowSwap                  698
UniswapV3Pool            687
Others                   561
DeFi Saver               372
1inch                    362
Curve                    208
Maverick                 123
Balancer                  91
GHO CCIP                  70
AirSwap                   69
MEV Bots                  64
Odos                      60
Gearbox                   56
Li.Fi                     50
Burn                      42
0x Exchange               33
Kyber Network             32
Pendle                    26
Notional                  21
GHO Stability Module      15
MultiSigs                 14
Name: to, dtype: int64

In [13]:
minter_txs_before.copy()['to'].apply(lambda x: contract_names.get(x, "Others")).value_counts()

UniswapV3Pool    1887
GhoAToken        1543
ParaSwap          794
1inch             560
Others            539
CowSwap           531
DeFi Saver        480
Balancer          169
AirSwap           143
GHO Staking       132
Curve              81
Maverick           66
0x Exchange        46
Kyber Network      26
MEV Bots           17
Li.Fi              12
MultiSigs           4
Odos                4
Name: to, dtype: int64

In [14]:
contracts = minter_txs.copy()
contracts['to'] = minter_txs.copy()['to'].apply(lambda x: contract_names.get(x, "Others"))
contracts[['to', 'value']].groupby('to').sum().sort_values('value', ascending=False)

,value
to,
GhoAToken,111223491.79
GHO Staking,62748484.39
CowSwap,50775019.65
ParaSwap,44010357.77
1inch,41575884.80
DeFi Saver,30647291.48
UniswapV3Pool,29530318.18
Balancer,27570367.13
Others,26267915.10


In [15]:
contracts_before = minter_txs_before.copy()
contracts_before['to'] = minter_txs_before.copy()['to'].apply(lambda x: contract_names.get(x, "Others"))
contracts_before[['to', 'value']].groupby('to').sum().sort_values('value', ascending=False)

,value
to,
GhoAToken,53913195.09
UniswapV3Pool,26212501.76
ParaSwap,22472774.30
Balancer,22392130.90
1inch,18496887.93
DeFi Saver,14085061.12
CowSwap,11803139.97
Others,8974284.47
Maverick,8513873.92


In [16]:
contracts_after = minter_txs_after.copy()
contracts_after['to'] = minter_txs_after.copy()['to'].apply(lambda x: contract_names.get(x, "Others"))
contracts_after[['to', 'value']].groupby('to').sum().sort_values('value', ascending=False)

,value
to,
GhoAToken,57310296.70
GHO Staking,54911119.79
CowSwap,38971879.68
1inch,23078996.87
ParaSwap,21537583.47
Others,17293630.63
DeFi Saver,16562230.36
Burn,7501864.43
MultiSigs,6378351.73


### Sent to EOAs

In [17]:
other_addresses = [user for user in minter_txs['to'] if user not in contract_names]
len(set(other_addresses))

411

In [18]:
import asyncio
from web3 import Web3
import concurrent
from tqdm import tqdm

provider = os.environ.get('WEB3_PROVIDER_URL')
w3 = Web3(Web3.HTTPProvider(provider))

def check_address_type(address):
    code = w3.eth.get_code(Web3.to_checksum_address(address))
    return address, len(code) > 0

def process_batch(addresses):
    return [check_address_type(addr) for addr in addresses]

def parallel_address_check(addresses, batch_size=20):
    eoa_count = 0
    contract_count = 0
    results = []

    with concurrent.futures.ThreadPoolExecutor() as executor:
        batches = [addresses[i:i + batch_size] for i in range(0, len(addresses), batch_size)]
        futures = [executor.submit(process_batch, batch) for batch in batches]

        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
            batch_results = future.result()
            for address, is_contract in batch_results:
                if is_contract:
                    contract_count += 1
                else:
                    eoa_count += 1
                results.append((address, 'Contract' if is_contract else 'EOA'))

    return results, eoa_count, contract_count

results, eoa_count, contract_count = parallel_address_check(list(set(other_addresses)))

print(f"EOAs: {eoa_count}")
print(f"Contracts: {contract_count}")

100%|████████████████████████████████████████████████████████████████████████████████| 21/21 [00:16<00:00,  1.30it/s]

EOAs: 299
Contracts: 112


In [26]:
eoas = [add for add, ctype in results if ctype == 'EOA']
not_eoas = [add for add, ctype in results if ctype != 'EOA']
print("Sent to EOAs:", sum(minter_txs[minter_txs['to'].isin(eoas)]['value']))
print("Sent to Other Contracts:", sum(minter_txs[minter_txs['to'].isin(not_eoas)]['value']))

Sent to EOAs: 18760899.259315927
Sent to Other Contracts: 7507015.841593794


In [24]:
len(not_eoas)

112

In [30]:
print("Sent to EOAs (before):", sum(minter_txs_before[minter_txs_before['to'].isin(eoas)]['value']))
print("Sent to Other Contracts (before):", sum(minter_txs_before[minter_txs_before['to'].isin(not_eoas)]['value']))
print("Sent to EOAs (before):", len(set(minter_txs_before[minter_txs_before['to'].isin(eoas)]['to'])))
print("Sent to Other Contracts (before):", len(set(minter_txs_before[minter_txs_before['to'].isin(not_eoas)]['to'].to_list())))

Sent to EOAs (before): 5455741.706843936
Sent to Other Contracts (before): 3518542.762939007
Sent to EOAs (before): 182
Sent to Other Contracts (before): 64


In [29]:
print("Sent to EOAs (after):", sum(minter_txs_after[minter_txs_after['to'].isin(eoas)]['value']))
print("Sent to Other Contracts (after):", sum(minter_txs_after[minter_txs_after['to'].isin(not_eoas)]['value']))
print("Sent to EOAs (after):", len(set(minter_txs_after[minter_txs_after['to'].isin(eoas)]['to'])))
print("Sent to Other Contracts (after):", len(set(minter_txs_after[minter_txs_after['to'].isin(not_eoas)]['to'].to_list())))

Sent to EOAs (after): 13305157.552471993
Sent to Other Contracts (after): 3988473.0786547903
Sent to EOAs (after): 134
Sent to Other Contracts (after): 62


In [58]:
main_protocols = ['GhoAToken', 'GHO Stability Module', 'Odos', 'UniswapV3Pool', 'ParaSwap', 'Gearbox', 'Balancer', '1inch', 'DeFi Saver', 'CowSwap', 'Maverick', 'GHO Staking', '0x Exchange', 'Curve', 'AirSwap']

In [48]:
fo_value_before.keys()

dict_keys(['GhoAToken', 'UniswapV3Pool', 'ParaSwap', 'Balancer', '1inch', 'DeFi Saver', 'CowSwap', 'Others', 'Maverick', 'GHO Staking', 'MEV Bots', 'MultiSigs', '0x Exchange', 'Curve', 'AirSwap', 'Kyber Network', 'Odos', 'Li.Fi', 'Sent to other EOA', 'Held in wallet'])

In [59]:
fo_value_before = contracts_before[['to', 'value']].groupby('to').sum().sort_values('value', ascending=False).to_dict()['value']
fo_value_before['Sent to other EOA'] = sum(minter_txs_before[minter_txs_before['to'].isin(eoas)]['value'])
fo_value_before["Others"] = sum(minter_txs_before[minter_txs_before['to'].isin(not_eoas)]['value'])
fo_value_before["Held in wallet"] = sum(df[df['to'].isin(hoarders) & (df['blockTimestamp'].astype(int) <= 1710421200) & (df['from'] == '0x0000000000000000000000000000000000000000')]['value'].astype(float)) * 1e-18

value_before = {}
value_before['Others'] = 0
for k, v in fo_value_before.items():
    if k in main_protocols:
        value_before[k] = v
    else:
        value_before['Others'] += v
        
{k: v / sum(value_before.values()) * 100 for k, v in value_before.items()}

{'Others': 7.0601173794268774,
 'GhoAToken': 26.368345670785526,
 'UniswapV3Pool': 12.820243841905556,
 'ParaSwap': 10.99118462280943,
 'Balancer': 10.95174283426872,
 '1inch': 9.046622707243571,
 'DeFi Saver': 6.88884715156559,
 'CowSwap': 5.772784830645803,
 'Maverick': 4.164041290106433,
 'GHO Staking': 3.8331680848257284,
 '0x Exchange': 0.828358069569634,
 'Curve': 0.7109441547486193,
 'AirSwap': 0.38759545085423397,
 'Odos': 0.1760039112442538}

In [61]:
fo_value_after = contracts_after[['to', 'value']].groupby('to').sum().sort_values('value', ascending=False).to_dict()['value']
fo_value_after['Sent to other EOA'] = sum(minter_txs_after[minter_txs_after['to'].isin(eoas)]['value'])
fo_value_after["Others"] = sum(minter_txs_after[minter_txs_after['to'].isin(not_eoas)]['value'])
fo_value_after["Held in wallet"] = sum(df[df['to'].isin(hoarders) & (df['blockTimestamp'].astype(int) <= 1710421200) & (df['from'] == '0x0000000000000000000000000000000000000000')]['value'].astype(float)) * 1e-18

value_after = {}
value_after['Others'] = 0
for k, v in fo_value_after.items():
    if k in main_protocols:
        value_after[k] = v
    else:
        value_after['Others'] += v
        
{k: v / sum(value_after.values()) * 100 for k, v in value_after.items()}

{'Others': 14.814680487037442,
 'GhoAToken': 19.903342158243376,
 'GHO Staking': 19.07012995009374,
 'CowSwap': 13.534577563998418,
 '1inch': 8.015124642920433,
 'ParaSwap': 7.479805859885773,
 'DeFi Saver': 5.7519112069062945,
 'Maverick': 2.041217008408768,
 'Curve': 2.027662326331488,
 'GHO Stability Module': 1.9164801975378973,
 'Balancer': 1.7983541047690292,
 'Odos': 1.1703255030174986,
 'UniswapV3Pool': 1.1522473141960305,
 'Gearbox': 1.1304801500603665,
 '0x Exchange': 0.12273346375545544,
 'AirSwap': 0.0709280628379815}

In [63]:
fo_count_before = minter_txs_before.copy()['to'].apply(lambda x: contract_names.get(x, "Others")).value_counts().to_dict()
fo_count_before['Sent to other EOA'] = len(set(minter_txs_before[minter_txs_before['to'].isin(eoas)]['to']))
fo_count_before['Others'] = len(set(minter_txs_before[minter_txs_before['to'].isin(not_eoas)]['to'].to_list()))
fo_count_before['Held in wallet'] = len(df[df['to'].isin(hoarders) & (df['blockTimestamp'].astype(int) <= 1710421200) & (df['from'] == '0x0000000000000000000000000000000000000000')]['value'])

count_before = {}
count_before['Others'] = 0
for k, v in fo_count_before.items():
    if k in main_protocols:
        count_before[k] = v
    else:
        count_before['Others'] += v
        
{k: v / sum(count_before.values()) * 100 for k, v in count_before.items()}

{'Others': 5.602816075095336,
 'UniswapV3Pool': 27.676738046347904,
 'GhoAToken': 22.63127016720446,
 'ParaSwap': 11.645643883836902,
 '1inch': 8.213552361396303,
 'CowSwap': 7.788207685538281,
 'DeFi Saver': 7.0401877383396885,
 'Balancer': 2.478732766207099,
 'AirSwap': 2.097389263713699,
 'GHO Staking': 1.9360516280434146,
 'Curve': 1.1880316808448226,
 'Maverick': 0.9680258140217073,
 '0x Exchange': 0.6746846582575535,
 'Odos': 0.05866823115283074}

In [64]:
fo_count_after = minter_txs_after.copy()['to'].apply(lambda x: contract_names.get(x, "Others")).value_counts().to_dict()
fo_count_after['Sent to other EOA'] = len(set(minter_txs_after[minter_txs_after['to'].isin(eoas)]['to']))
fo_count_after['Others'] = len(set(minter_txs_after[minter_txs_after['to'].isin(not_eoas)]['to'].to_list()))
fo_count_after['Held in wallet'] = len(df[df['to'].isin(hoarders) & (df['blockTimestamp'].astype(int) <= 1710421200) & (df['from'] == '0x0000000000000000000000000000000000000000')]['value'])

count_after = {}
count_after['Others'] = 0
for k, v in fo_count_after.items():
    if k in main_protocols:
        count_after[k] = v
    else:
        count_after['Others'] += v
        
{k: v / sum(count_after.values()) * 100 for k, v in count_after.items()}

{'Others': 8.728988498967857,
 'GhoAToken': 22.279563550575052,
 'GHO Staking': 17.472721910940724,
 'ParaSwap': 10.616337363609555,
 'CowSwap': 10.291949277499263,
 'UniswapV3Pool': 10.129755234444117,
 'DeFi Saver': 5.485107637864937,
 '1inch': 5.337658507814804,
 'Curve': 3.06694190504276,
 'Maverick': 1.8136242996166323,
 'Balancer': 1.3417870834562076,
 'AirSwap': 1.0173989973459157,
 'Odos': 0.8846947803007963,
 'Gearbox': 0.8257151282807431,
 '0x Exchange': 0.48658212916543786,
 'GHO Stability Module': 0.22117369507519907}

In [196]:
minter_full_txs = df[df['from'].isin(minters)]

We're only going to check DEX trades

In [198]:
dexes = ['1inch', 'Odos', 'Curve', 'CowSwap', 'ParaSwap', 'Maverick', 'Balancer', 'UniswapV3Pool', '0x Exchange', 'Kyber Network', 'Li.Fi', 'Others', 'MultiSigs', 'MEV Bots']
minter_full_txs['destination'] =  minter_full_txs['to'].apply(lambda x: contract_names.get(x, "Others"))
dex_txs = minter_full_txs[minter_full_txs['destination'].isin(dexes)]
dex_txs

/tmp/ipykernel_103243/1021720791.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,from,to,value,transferIndex,borrowCap,supplyCap,totalVariableDebt,variableBorrowRate,blockNumber,blockTimestamp,transactionHash,mint,destination
10,0x79603115df2ba00659adc63192325cf104ca529c,0x5459ba81d4b375a7b2e34aa83edb1e18c4dc181e,155934142056500000000,10,0,0,81343918969610398129157,15000000000000000000000000,17699326,1689430739,0x9e71943828922409e045ff5c9b9011925d542de62a7df65d60c9a135c24bc8d7,False,Others
18,0xd2eeff73117c86c14f11a6052620848f8dd6e0c8,0xc0268fdbc7f526b43cdbf34e7529b7da0d06dd55,28256317265407911651909,18,0,0,277080953174315786866668,15000000000000000000000000,17699363,1689431195,0xc2c13363f315249fc3481b0c94397edbc15269b87bad6322cf35b5299707bec6,False,Others
38,0xd2eeff73117c86c14f11a6052620848f8dd6e0c8,0xc0268fdbc7f526b43cdbf34e7529b7da0d06dd55,16812655822481809110230,38,0,0,278461016597257517749073,15000000000000000000000000,17699401,1689431675,0x2f8ebe2e50c7b6ddbbaab901e68936d166aef8a0140646dd3933d4962c3d19a8,False,Others
50,0x9e9fbeb970383a43d6fba10af4bc0a96e567f83a,0x8e5c57e69ff1a5a34a0358a260d73981f38e924c,100000000000000000000,50,0,0,281362106879226153725806,15000000000000000000000000,17699435,1689432107,0x8f0f230cabf7e21a954886310b70dd0b7c1da1799ff4aced441fee00b9f16e48,False,Others
55,0x2e21f5d32841cf8c7da805185a041400bf15f21a,0xc0268fdbc7f526b43cdbf34e7529b7da0d06dd55,12977707562003048338489,55,0,0,281162188722374181974933,15000000000000000000000000,17699485,1689432719,0x65ad012bb84f96890b88a8cbc1d1aa889333bab85ef27d4db038c5bd0e2151aa,False,Others
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82022,0x0a9972b72910b8668755d0dc016ecc68d1cbbcca,0x16c6521dff6bab339122a0fe25a9116693265353,5000000000000000000000,82022,125000000,0,111471857694936162785433874,60000000000000000000000000,20682692,1725518867,0x84b13ede19fa90a3e9c92e514b89d13d54542d28ddeb2f7f13d93ee4cc2d9a0c,False,Curve
82065,0xd6c1f6cd5134568979f3c8cf41ed2332cb6b4a74,0x1231deb6f5749ef6ce6943a275a1d3e7486f4eae,2017588478833170819005,82065,125000000,0,111470318026089831961216830,60000000000000000000000000,20683212,1725525167,0x9427f8f1445d4931c5e76ff3dcce55b9639b3a63a897230bdb9db9248977ee86,False,Li.Fi
82069,0x4f0a01badaa24f762cee620883f16c4460c06be0,0x28104d4f703ee5b5011cefe106f54efd56f33f95,167000999250672397414184,82069,125000000,0,111470450365358614119627153,60000000000000000000000000,20683263,1725525791,0x6d58e97b207f484b57026614a034aef0e1518069f03cd3a10a6dd45c1b43e2e3,False,Odos
82078,0x4f0a01badaa24f762cee620883f16c4460c06be0,0x5c95d4b1c3321cf898d25949f41d50be2db5bc1d,130421901502327359231,82078,125000000,0,111470554709892853590148427,60000000000000000000000000,20683304,1725526283,0x8e8e60ab09cd4a9ffcebf296395599fb7e08d34c0da23b3df72e1c6b20408a79,False,UniswapV3Pool


### Parse token transfers
We're going to look for ERC20 transfers going TO the dumper address in the same transaction and we'll fetch the symbol for those

In [296]:
from web3.exceptions import ContractLogicError
import aiohttp
import pandas as pd

w3 = Web3(Web3.HTTPProvider(provider))

ERC20_ABI = [
    {
        "constant": True,
        "inputs": [],
        "name": "symbol",
        "outputs": [{"name": "", "type": "string"}],
        "type": "function"
    },
    {
        "anonymous": False,
        "inputs": [
            {"indexed": True, "name": "from", "type": "address"},
            {"indexed": True, "name": "to", "type": "address"},
            {"indexed": False, "name": "value", "type": "uint256"}
        ],
        "name": "Transfer",
        "type": "event"
    }
]

LP_EVENT_TOPICS = [
    '0x3067048beee31b25b2f1681f88dac838c8bba36af25bfb2b7cf7473a5847e35f' # IncreaseLiquidity UniV3
    '0x26f55a85081d24974e85c6c00045d0f0453991e95873f52bff0d21af4079a768' # AddLiquidity Curve
]

symbols = {'0x9f8F72aA9304c8B593d555F12eF6589cC3A579A2'.lower(): 'MKR'}
decimals = {'USDT': 1000000,
 'DAI': 1000000000000000000,
 'crvUSD': 1000000000000000000,
 'USDC': 1000000,
 'mkUSD': 1000000000000000000,
 'FRAX': 1000000000000000000,
 'stUSD': 1000000000000000000,
 'USDe': 1000000000000000000,
 'PYUSD': 1000000,
 'LUSD': 1000000000000000000,
 'sDAI': 1000000000000000000,
 'sUSD': 1000000000000000000,
 'rgUSD': 1000000000000000000,
 'DOLA': 1000000000000000000,
 'fxUSD': 1000000000000000000,
 'USDP': 1000000000000000000,
 'USD3': 1000000000000000000,
 'USD0': 1000000000000000000,
 'sUSDe': 1000000000000000000,
 'GHO': 1000000000000000000,
 'GUSD': 100}


def get_token_symbol(token_address):
    contract = w3.eth.contract(address=token_address, abi=ERC20_ABI)
    try:
        symbol = contract.functions.symbol().call()
        return symbol
    except ContractLogicError:
        return "Unknown"

def process_transaction(tx_hash, target_address, destination, value_out):
    try:
        receipt = w3.eth.get_transaction_receipt(tx_hash)
    except Exception as e:
        print(e)
        return {}
    events = []

    added_logs = 0
    current_events = []
    for log in receipt.logs:
        if len(log['topics']) in [3,4] and log['topics'][0].hex() == Web3.keccak(text="Transfer(address,address,uint256)").hex():
            to_address = '0x' + log['topics'][2].hex()[-40:]
            if to_address.lower() == target_address.lower():
                from_address = '0x' + log['topics'][1].hex()[-40:]
                token_address = log['address']
                try:
                    value = int.from_bytes(log['data'], byteorder='big')
                except ValueError:
                    value = 0
                    print(f"Invalid value data for tx_hash: {tx_hash}")
                
                if token_address.lower() in symbols:
                    symbol = symbols[token_address.lower()]
                else:
                    try:
                        symbol = get_token_symbol(token_address)
                        symbols[token_address.lower()] = symbol
                    except Exception as e:
                        print(f'{e}: {token_address} : {tx_hash}')
                        symbol = "Missing"

                real_value_out = value_out
                if symbol in decimals:
                    real_value_out = value / decimals[symbol]
                    if real_value_out > value_out * 1.15:
                        real_value_out = 0
                    
                    if (symbol == 'GHO') and (real_value_out > value_out / 2):
                        real_value_out = 0
                    
                elif added_logs > 0:
                    real_value_out = 0
                    
                events.append({
                    'tx_hash': tx_hash,
                    'token_address': token_address,
                    'symbol': symbol,
                    'value_out': real_value_out,
                    'value_in': value,
                    'from': from_address,
                    'to': to_address,
                    'destination': destination,
                })
                added_logs += 1
    
    return events

def process_batch(batch):
    results = []
    for tx_hash, target_address, destination, value in batch:
        results.extend(process_transaction(tx_hash, target_address, destination, value))
    return results

def parallel_transaction_processing(tx_hashes, target_addresses, destinations, values, batch_size=20):
    all_events = []

    tx_address_pairs = list(zip(tx_hashes, target_addresses, destinations, values))

    with concurrent.futures.ThreadPoolExecutor() as executor:
        batches = [tx_address_pairs[i:i + batch_size] for i in range(0, len(tx_address_pairs), batch_size)]
        futures = [executor.submit(process_batch, batch) for batch in batches]

        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
            batch_events = future.result()
            all_events.extend(batch_events)

    return pd.DataFrame(all_events)

tx_hashes = dex_txs['transactionHash'].to_list()
target_addresses = dex_txs['from'].to_list()
destinations = dex_txs['destination'].to_list()
values = (dex_txs['value'].astype(float) * 1e-18).to_list()

result_df = parallel_transaction_processing(tx_hashes, target_addresses, destinations, values)

  4%|██▉                                                                            | 16/422 [00:05<01:24,  4.82it/s]

Transaction with hash: '0x4d4a058008bef8847cf95f6f97e545038e6e7a1b4206bb736eeb0fb485b3d50c' not found.


  4%|███▏                                                                           | 17/422 [00:07<03:53,  1.74it/s]

Transaction with hash: '0xa24cb747b3c879c408f07daccd08196a9bc6053485fe3ad4c265d29fa5da2cc0' not found.


  6%|█████                                                                          | 27/422 [00:09<01:19,  4.96it/s]

...

 64%|██████████████████████████████████████████████████▎                           | 272/422 [01:24<00:51,  2.91it/s]

Transaction with hash: '0x187cb1985058e7eb0c04818e05d9fd25f4aff7501a4dd27005e9880b9c047019' not found.


 66%|███████████████████████████████████████████████████▊                          | 280/422 [01:26<00:50,  2.82it/s]

Transaction with hash: '0x1d54470b51deb309fc9d3eebf2c5596f5da602d4991053671a78acf9452107e0' not found.


100%|██████████████████████████████████████████████████████████████████████████████| 422/422 [02:07<00:00,  3.32it/s]


In [131]:
pd.set_option('display.max_colwidth', None)

In [297]:
dex_txs['value'] = dex_txs['value'].astype(float)
(dex_txs[['destination', 'value']].groupby('destination').sum() * 1e-18).sort_values('value', ascending=False)

/tmp/ipykernel_103243/2050839116.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,value
destination,
CowSwap,50775019.65
ParaSwap,44010357.77
1inch,41575884.80
UniswapV3Pool,29530318.18
Balancer,27570367.13
Others,26267915.10
Maverick,14391417.07
MultiSigs,8301681.69
Curve,7292122.62


In [298]:
result_df

,tx_hash,token_address,symbol,value_out,value_in,from,to,destination
0,0x84b396c7fad17e16bb22ede6978340d240a1d61c8f936e49f5f679a563923657,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,UNI-V3-POS,146053.25,0,0x0000000000000000000000000000000000000000,0x36c4bd54d54dd898c242f5f634f5d0cef3be2a8a,UniswapV3Pool
1,0x2cfac29f66bffd5bac296c8b832e631589e2e185e277015b04e7d25a3f26ebda,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,UNI-V3-POS,64955.25,0,0x0000000000000000000000000000000000000000,0x36c4bd54d54dd898c242f5f634f5d0cef3be2a8a,UniswapV3Pool
2,0x072ac24481a9a8220932a0f7f916593124203708314b2935265f0953d8f7f37f,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,USDC,2462.77,2462772242,0x5c95d4b1c3321cf898d25949f41d50be2db5bc1d,0xfb13776ac0a308919fc8a4eb9e8f7eac7d8f1bda,UniswapV3Pool
3,0x4e546baa664234ad58960ba1fd78fb2dc5be771ddd0bce63c5e43317452f91d4,0x7D1AfA7B718fb893dB30A3aBc0Cfc608AaCfeBB0,MATIC,1200.00,1573729039542563443672,0x290a6a7460b308ee3f19023d2d00de604bcf5b42,0x698373ba1186edb9f9dd8ed46fb69f1580015153,UniswapV3Pool
4,0x97cf14deadbd3230a3394e9458f5c0565d1e3c4becfb2cd57b982cc382708225,0x6B175474E89094C44Da98b954EedeAC495271d0F,DAI,99444.94,99444935608140725847507,0x5777d92f208679db4b9778590fa3cab3ac9e2168,0x3cd361345df2a61d2fa0e2d1692da4404f1a4a11,UniswapV3Pool
...,...,...,...,...,...,...,...,...
9845,0xb3c4f953af0c855cc35a8d350317135fa77376e1c344bb53baa74f6881172588,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,WETH,0.00,286548730419637726,0xc351e45db65d68585e180795537563d33b3716e7,0x60a4dadea54fd242d11462667598a73473543542,ParaSwap
9846,0xb3c4f953af0c855cc35a8d350317135fa77376e1c344bb53baa74f6881172588,0x4d5F47FA6A74757f35C14fD3a6Ef8E3C9BC514E8,aEthWETH,0.00,286548730419637726,0x0000000000000000000000000000000000000000,0x60a4dadea54fd242d11462667598a73473543542,ParaSwap
9847,0xb3c4f953af0c855cc35a8d350317135fa77376e1c344bb53baa74f6881172588,0x786dBff3f1292ae8F92ea68Cf93c30b34B1ed04B,variableDebtEthGHO,0.00,687555302558804718771,0x0000000000000000000000000000000000000000,0x60a4dadea54fd242d11462667598a73473543542,ParaSwap
9848,0xb3c4f953af0c855cc35a8d350317135fa77376e1c344bb53baa74f6881172588,0x40D16FC0246aD3160Ccc09B8D0D3A2cD28aE6C2f,GHO,0.00,687529058683414583175,0x0000000000000000000000000000000000000000,0x60a4dadea54fd242d11462667598a73473543542,ParaSwap


In [299]:
result_df[result_df['destination'] == 'ParaSwap'].sort_values('value_out')

,tx_hash,token_address,symbol,value_out,value_in,from,to,destination
9848,0xb3c4f953af0c855cc35a8d350317135fa77376e1c344bb53baa74f6881172588,0x40D16FC0246aD3160Ccc09B8D0D3A2cD28aE6C2f,GHO,0.00,687529058683414583175,0x0000000000000000000000000000000000000000,0x60a4dadea54fd242d11462667598a73473543542,ParaSwap
3939,0x78f2277470ca87920e6d9a93b67ab673a259d9e7745475d184902660387fd6b7,0x40D16FC0246aD3160Ccc09B8D0D3A2cD28aE6C2f,GHO,0.00,431217874505427672802773,0x0000000000000000000000000000000000000000,0x8761e0370f94f68db8eaa731f4fc581f6ad0bd68,ParaSwap
3938,0x72028323caad51a77d6865b49fc0212d76114769d6acae09e009bded8e81977e,0x40D16FC0246aD3160Ccc09B8D0D3A2cD28aE6C2f,GHO,0.00,312755738944697813985012,0x0000000000000000000000000000000000000000,0x8761e0370f94f68db8eaa731f4fc581f6ad0bd68,ParaSwap
6638,0x2e43194f5300017d85c67b2781a2356f17692e6406a005185c3f7b279ccd6242,0x40D16FC0246aD3160Ccc09B8D0D3A2cD28aE6C2f,GHO,0.00,15945030619558558197236,0x0000000000000000000000000000000000000000,0x8761e0370f94f68db8eaa731f4fc581f6ad0bd68,ParaSwap
6643,0x3422c7659b4054cff39d4e5d2a1ff48ebdb903174fd933823cbf3fa3503ebb1f,0x40D16FC0246aD3160Ccc09B8D0D3A2cD28aE6C2f,GHO,0.00,752482628133602876652546,0x0000000000000000000000000000000000000000,0x8761e0370f94f68db8eaa731f4fc581f6ad0bd68,ParaSwap
...,...,...,...,...,...,...,...,...
7343,0x1980a77d5fff34270eda807141c21984da150e15935b83611931e3f97d42c8e6,0xdAC17F958D2ee523a2206206994597C13D831ec7,USDT,538065.41,538065407766,0xc351e45db65d68585e180795537563d33b3716e7,0x43303c5b97d2858557610acdb28992ab709c26b2,ParaSwap
6528,0xadf5b74dc0365c9d315879b18edaa911e4ff6d39d15e54c8b0593274a5883702,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,USDC,550000.00,550000000000,0xdef171fe48cf0115b1d80b88dc8eab59176fee57,0x8761e0370f94f68db8eaa731f4fc581f6ad0bd68,ParaSwap
5288,0x68aaae60f64da448d63f6477e788686ca532d9e9bc5f3aa9b56db32900b676ff,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,USDC,577490.96,577490957363,0xdef171fe48cf0115b1d80b88dc8eab59176fee57,0x8761e0370f94f68db8eaa731f4fc581f6ad0bd68,ParaSwap
6347,0x20844888555292db31ec00cc66fe8793dad0128c35268dbcfa5fb908bfc297d7,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,USDC,750000.00,750000000000,0xdef171fe48cf0115b1d80b88dc8eab59176fee57,0x8761e0370f94f68db8eaa731f4fc581f6ad0bd68,ParaSwap


In [300]:
result_df[['symbol', 'destination', 'value_out']].groupby(['destination', 'symbol']).sum()

value_out
destination   symbol                          
0x Exchange   AAVE                    28840.86
              BTRFLY                    100.43
              DAI                    109085.01
              DMT                     10000.00
              GHO                         0.00
...                                        ...
UniswapV3Pool swETH                     100.00
              variableDebtEthGHO          0.00
              variableDebtEthcrvUSD       0.00
              wTAO                    15000.00
              wstETH                 454956.13

[485 rows x 1 columns]

In [301]:
tokens_df = result_df[['symbol', 'destination', 'value_out']].groupby(['destination', 'symbol']).sum()

In [302]:
tokens_df.loc['UniswapV3Pool']

,value_out
symbol,
AAVE,585396.56
ACX,17500.00
ANKR,1242.25
ARB,8750.00
ARES,73.21
...,...
swETH,100.00
variableDebtEthGHO,0.00
variableDebtEthcrvUSD,0.00


In [303]:
LP_SYMBOLS = {'UNI-V3-POS','ECLP-GHO-GYD',
 'ECLP-GHO-USDC',
              'GHOBTCwstE',
 'ECLP-GHO-USDC-2',
 'GHO-3POOL-BPT',
 'GHO/bb-a-USD',
              'auraGHO/USDT/USDC-vault',
 'MBP-GHO-USDC-15-R1',
 'MBP-GHO-USDC-18-R1',
 'MBP-GHO-USDC-21',
              'MBP-GHO-USD0-13-R1',
              
 'MBP-GHO-USDC-21-R1',
              'SPT-PT/IBT-f',
 'MBP-GHO-USDC-23-R1',
 'MBP-GHO-USDC-4-R1',
 'UNI-V3-POS',
 'USDC-DAI-USDT','GHOcrvUSD-gauge',
              'MPv2', 'mooFxConvexGHO-fxUSD',
 'bb-a-USD',
 'crvUSDGHO-f',
              'GHO/bb-a-USD',  'GHO/LUSD', 'GHO/USDT/USDC', 'GHO/bb-a-USD', 'GHO/USDT/USDC', 'GHOUSDe', 'fxUSDGHO', 'crvUSDGHO-f', 'GHOcrvUSD', 'GHOUSDe', '80wstETH/20GHO'}

ETH_AND_LSTS = {'boxETH',
 'cbETH',
 'ezETH',
 'osETH',
 'rETH',
 'ETH2X',
 'sfrxETH',
 'stETH',
 'swETH',
 'ETHx',
 'WETH',
 'weETH',
 'wstETH'}

BTC = {'LBTC', 'WBTC', 'tBTC'}

OTHER_STABLES = { 'mkUSD',
 'rgUSD',
 'sUSDe', 
 'fxUSD',
                 'sUSD',
 'stUSD', 
 'bb-a-USD',
 'USDP', 
 #'PYUSD',
 'DOLA',
 'USD0',
 'USD3',
 'USDA',
 'GUSD',
                 'sDAI'
 }

A_TOKENS = {
 'aEthAAVE',
 'aEthDAI',
 'aEthUSDC',
    'aEthETHx',
 'aEthUSDT',
 'aEthUSDe',
 'aEthWBTC',
 'aEthWETH',
 'aEthcrvUSD',
 'aEthrETH',
 'aEthwstETH'}

GHO = {'pGHO', 'YT-sw-stkGHO-1732492824', 'GHO-agg', 'nGHO', 'pdGHO', 'PT-sw-stkGHO-1732492824'}

VAR_DEBT_TOKENS = {'variableDebtEthGHO', 'variableDebtEthUSDC', 'variableDebtEthcrvUSD', 'variableDebtEthDAI', 'variableDebtEthLDO'}

MAJORS = {'GHO', 'USDC', 'USDT', 'DAI', 'AAVE', 'LINK', 'LUSD', 'crvUSD', 'USDe', 'FRAX', 'PYUSD'}


In [304]:
result_df[~result_df['symbol'].isin(VAR_DEBT_TOKENS | GHO | A_TOKENS | BTC | OTHER_STABLES | ETH_AND_LSTS | LP_SYMBOLS | MAJORS)][['symbol', 'value_out']].groupby('symbol').sum().sort_values('value_out', ascending=False)[:50]

,value_out
symbol,
MKR,284650.37
CRV,249457.16
wTAO,139500.00
LDO,116852.03
R,87570.29
AURA,79700.00
UNI,67802.90
DYDX,53030.00
CNC,50000.00


### Relabelling our symbols

In [305]:
def relabel(label):
    if label in A_TOKENS:
        return 'aTokens'
    elif label in MAJORS:
        return label
    elif label in VAR_DEBT_TOKENS:
        return 'Variable debt tokens'
    elif label in OTHER_STABLES:
        return 'Other Stablecoins'
    elif label in BTC:
        return 'BTC'
    elif label in ETH_AND_LSTS:
        return 'ETH/LSTs/LRTs'
    elif label in GHO:
        return 'GHO Yield Aggregation'
    elif label in LP_SYMBOLS:
        return 'Providing Liquidity'
    return 'Other crypto'

label_df = result_df.copy()
label_df['symbol'] = label_df['symbol'].apply(relabel)

label_df

,tx_hash,token_address,symbol,value_out,value_in,from,to,destination
0,0x84b396c7fad17e16bb22ede6978340d240a1d61c8f936e49f5f679a563923657,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,Providing Liquidity,146053.25,0,0x0000000000000000000000000000000000000000,0x36c4bd54d54dd898c242f5f634f5d0cef3be2a8a,UniswapV3Pool
1,0x2cfac29f66bffd5bac296c8b832e631589e2e185e277015b04e7d25a3f26ebda,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,Providing Liquidity,64955.25,0,0x0000000000000000000000000000000000000000,0x36c4bd54d54dd898c242f5f634f5d0cef3be2a8a,UniswapV3Pool
2,0x072ac24481a9a8220932a0f7f916593124203708314b2935265f0953d8f7f37f,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,USDC,2462.77,2462772242,0x5c95d4b1c3321cf898d25949f41d50be2db5bc1d,0xfb13776ac0a308919fc8a4eb9e8f7eac7d8f1bda,UniswapV3Pool
3,0x4e546baa664234ad58960ba1fd78fb2dc5be771ddd0bce63c5e43317452f91d4,0x7D1AfA7B718fb893dB30A3aBc0Cfc608AaCfeBB0,Other crypto,1200.00,1573729039542563443672,0x290a6a7460b308ee3f19023d2d00de604bcf5b42,0x698373ba1186edb9f9dd8ed46fb69f1580015153,UniswapV3Pool
4,0x97cf14deadbd3230a3394e9458f5c0565d1e3c4becfb2cd57b982cc382708225,0x6B175474E89094C44Da98b954EedeAC495271d0F,DAI,99444.94,99444935608140725847507,0x5777d92f208679db4b9778590fa3cab3ac9e2168,0x3cd361345df2a61d2fa0e2d1692da4404f1a4a11,UniswapV3Pool
...,...,...,...,...,...,...,...,...
9845,0xb3c4f953af0c855cc35a8d350317135fa77376e1c344bb53baa74f6881172588,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,ETH/LSTs/LRTs,0.00,286548730419637726,0xc351e45db65d68585e180795537563d33b3716e7,0x60a4dadea54fd242d11462667598a73473543542,ParaSwap
9846,0xb3c4f953af0c855cc35a8d350317135fa77376e1c344bb53baa74f6881172588,0x4d5F47FA6A74757f35C14fD3a6Ef8E3C9BC514E8,aTokens,0.00,286548730419637726,0x0000000000000000000000000000000000000000,0x60a4dadea54fd242d11462667598a73473543542,ParaSwap
9847,0xb3c4f953af0c855cc35a8d350317135fa77376e1c344bb53baa74f6881172588,0x786dBff3f1292ae8F92ea68Cf93c30b34B1ed04B,Variable debt tokens,0.00,687555302558804718771,0x0000000000000000000000000000000000000000,0x60a4dadea54fd242d11462667598a73473543542,ParaSwap
9848,0xb3c4f953af0c855cc35a8d350317135fa77376e1c344bb53baa74f6881172588,0x40D16FC0246aD3160Ccc09B8D0D3A2cD28aE6C2f,GHO,0.00,687529058683414583175,0x0000000000000000000000000000000000000000,0x60a4dadea54fd242d11462667598a73473543542,ParaSwap


In [306]:
label_df[['symbol', 'value_out']].groupby('symbol').sum().sort_values('value_out', ascending=False)

,value_out
symbol,
USDC,86032402.67
USDT,39470296.27
Providing Liquidity,31842600.80
DAI,15226772.02
ETH/LSTs/LRTs,11346968.54
Other Stablecoins,8843945.87
BTC,8559649.38
Variable debt tokens,4023350.15
Other crypto,1938004.65


In [314]:
grouped = label_df[['symbol', 'destination', 'value_out']].groupby(['destination', 'symbol']).sum()
grouped = grouped.rename(columns={'value_out': 'sum'})
grouped['percentage'] = grouped.groupby(level=0).apply(lambda x: x / x.sum() * 100)
grouped = grouped.reset_index()
grouped = grouped.set_index(['destination', 'symbol'])
grouped

/tmp/ipykernel_103243/3233207668.py:3: FutureWarning:

Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)



sum  percentage
destination   symbol                                     
0x Exchange   AAVE                   28840.86        1.49
              BTC                   146410.82        7.56
              DAI                   109085.01        5.63
              ETH/LSTs/LRTs         266749.53       13.77
              GHO                        0.00        0.00
...                                       ...         ...
UniswapV3Pool USDT                 2898534.06       10.96
              USDe                     170.84        0.00
              Variable debt tokens       0.00        0.00
              aTokens               155989.03        0.59
              crvUSD                193975.92        0.73

[184 rows x 2 columns]

In [307]:
grouped = label_df[['destination', 'symbol', 'value_out']].groupby(['destination', 'symbol']).sum()
grouped

value_out
destination   symbol                         
0x Exchange   AAVE                   28840.86
              BTC                   146410.82
              DAI                   109085.01
              ETH/LSTs/LRTs         266749.53
              GHO                        0.00
...                                       ...
UniswapV3Pool USDT                 2898534.06
              USDe                     170.84
              Variable debt tokens       0.00
              aTokens               155989.03
              crvUSD                193975.92

[184 rows x 1 columns]

In [308]:
first_order = [{'from': 'Minting', 'to': k, 'amount': v} for k, v in first_order_flows.items()]

In [309]:
first_order

[{'from': 'Minting', 'to': 'GhoAToken', 'amount': 111223491.78571625},
 {'from': 'Minting', 'to': 'GHO Staking', 'amount': 62748484.38607395},
 {'from': 'Minting', 'to': 'CowSwap', 'amount': 50775019.65455372},
 {'from': 'Minting', 'to': 'ParaSwap', 'amount': 44010357.76625982},
 {'from': 'Minting', 'to': '1inch', 'amount': 41575884.80412208},
 {'from': 'Minting', 'to': 'DeFi Saver', 'amount': 30647291.484721772},
 {'from': 'Minting', 'to': 'UniswapV3Pool', 'amount': 29530318.18152685},
 {'from': 'Minting', 'to': 'Balancer', 'amount': 27570367.126080755},
 {'from': 'Minting', 'to': 'Others', 'amount': 7507015.841593794},
 {'from': 'Minting', 'to': 'Maverick', 'amount': 14391417.067078952},
 {'from': 'Minting', 'to': 'MultiSigs', 'amount': 8301681.6875084},
 {'from': 'Minting', 'to': 'Burn', 'amount': 7501864.433658},
 {'from': 'Minting', 'to': 'Curve', 'amount': 7292122.621455466},
 {'from': 'Minting', 'to': 'MEV Bots', 'amount': 6191507.644309231},
 {'from': 'Minting',
  'to': 'GHO St

In [310]:
second_order = []
for protocol, amount in first_order_flows.items():
    if protocol not in dexes:
        continue
    for token in grouped.loc[protocol].sort_values('value_out', ascending=False).index:
        second_order.append(
            {'from': protocol,
            'to': token,
            'amount': (grouped.loc[protocol,token].value_out)}
        )
second_order

[{'from': 'CowSwap', 'to': 'USDC', 'amount': 20118184.521228},
 {'from': 'CowSwap', 'to': 'USDT', 'amount': 13618830.602547001},
 {'from': 'CowSwap', 'to': 'BTC', 'amount': 4978048.34371197},
 {'from': 'CowSwap', 'to': 'DAI', 'amount': 3175591.2215765556},
 {'from': 'CowSwap', 'to': 'ETH/LSTs/LRTs', 'amount': 1199023.9071701076},
 {'from': 'CowSwap', 'to': 'Other crypto', 'amount': 697764.3246797635},
 {'from': 'CowSwap', 'to': 'FRAX', 'amount': 636481.9284981864},
 {'from': 'CowSwap', 'to': 'PYUSD', 'amount': 603908.200248},
 {'from': 'CowSwap', 'to': 'LINK', 'amount': 551811.2392530986},
 {'from': 'CowSwap', 'to': 'Other Stablecoins', 'amount': 322003.0887441111},
 {'from': 'CowSwap', 'to': 'AAVE', 'amount': 272712.8404730047},
 {'from': 'CowSwap', 'to': 'crvUSD', 'amount': 178068.38979904994},
 {'from': 'CowSwap', 'to': 'USDe', 'amount': 147090.92536851845},
 {'from': 'CowSwap', 'to': 'LUSD', 'amount': 90151.26874893496},
 {'from': 'CowSwap', 'to': 'aTokens', 'amount': 17226.7801343

In [315]:
second_order = []
for protocol, amount in first_order_flows.items():
    if protocol not in dexes:
        continue
    for token in grouped.loc[protocol].sort_values('percentage', ascending=False).index:
        second_order.append(
            {'from': protocol,
            'to': token,
            'amount': int(grouped.loc[protocol,token].percentage / 100 * amount)}
        )
second_order

[{'from': 'CowSwap', 'to': 'USDC', 'amount': 21917382},
 {'from': 'CowSwap', 'to': 'USDT', 'amount': 14836782},
 {'from': 'CowSwap', 'to': 'BTC', 'amount': 5423242},
 {'from': 'CowSwap', 'to': 'DAI', 'amount': 3459588},
 {'from': 'CowSwap', 'to': 'ETH/LSTs/LRTs', 'amount': 1306254},
 {'from': 'CowSwap', 'to': 'Other crypto', 'amount': 760166},
 {'from': 'CowSwap', 'to': 'FRAX', 'amount': 693403},
 {'from': 'CowSwap', 'to': 'PYUSD', 'amount': 657916},
 {'from': 'CowSwap', 'to': 'LINK', 'amount': 601160},
 {'from': 'CowSwap', 'to': 'Other Stablecoins', 'amount': 350800},
 {'from': 'CowSwap', 'to': 'AAVE', 'amount': 297101},
 {'from': 'CowSwap', 'to': 'crvUSD', 'amount': 193993},
 {'from': 'CowSwap', 'to': 'USDe', 'amount': 160245},
 {'from': 'CowSwap', 'to': 'LUSD', 'amount': 98213},
 {'from': 'CowSwap', 'to': 'aTokens', 'amount': 18767},
 {'from': 'CowSwap', 'to': 'GHO', 'amount': 0},
 {'from': 'ParaSwap', 'to': 'USDC', 'amount': 27848070},
 {'from': 'ParaSwap', 'to': 'USDT', 'amount': 

In [311]:
label_df.drop_duplicates(['from', 'to'])['symbol'].value_counts().to_list()
grouped.loc['UniswapV3Pool'].sort_values('value_out', ascending=False)

,value_out
symbol,
USDC,9113821.74
Providing Liquidity,8109219.44
USDT,2898534.06
BTC,1526304.05
ETH/LSTs/LRTs,1516350.20
DAI,1170568.43
AAVE,585396.56
LINK,513424.44
Other crypto,480563.93


In [319]:
import json

with open('first_order_vol.json', 'w') as f:
    json.dump(sorted(first_order, key=lambda x: x['amount'], reverse=True), f, indent=2)

with open('second_order_vol.json', 'w') as f:
    json.dump(second_order, f, indent=2)

In [39]:
with open('first_order.json', 'r') as f:
    first_order = json.load(f)

with open('second_order.json', 'r') as f:
    second_order = json.load(f)

In [318]:
import plotly.graph_objects as go
import pandas as pd

def create_sankey(first_order, second_order):
    nodes = {}
    links = []
    
    def add_node(name):
        if name not in nodes:
            nodes[name] = len(nodes)
        return nodes[name]
    
    for item in first_order:
        source = add_node(item['from'])
        target = add_node(item['to'])
        links.append((source, target, item['amount']))
    
    for item in second_order:
        source = add_node(item['from'])
        target = add_node(item['to'])
        links.append((source, target, item['amount']))
    
    node_list = sorted(nodes.items(), key=lambda x: x[1])
    node_names = [name for name, _ in node_list]
    
    link_sources, link_targets, link_values = zip(*links)
    
    sankey = go.Sankey(
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=node_names,
            color="blue"
        ),
        link=dict(
            source=link_sources,
            target=link_targets,
            value=link_values
        )
    )
    
    layout = go.Layout(
        title_text="Stablecoin Flow Sankey Diagram",
        font_size=10
    )
    
    fig = go.Figure(data=[sankey], layout=layout)
    return fig

sankey_fig = create_sankey(first_order, second_order)

sankey_fig.write_html("sankey_plot.html")
from IPython.display import IFrame
IFrame(src="sankey_plot.html", width=900, height=600)

In [48]:
import plotly.graph_objects as go

def create_sankey(first_order, second_order):
    nodes = {}
    links = []
    
    def add_node(name):
        if name not in nodes:
            nodes[name] = len(nodes)
        return nodes[name]
    
    for item in first_order:
        source = add_node(item['from'])
        target = add_node(item['to'])
        links.append((source, target, item['amount']))
    
    for item in second_order:
        source = add_node(item['from'])
        target = add_node(item['to'])
        links.append((source, target, item['amount']))
    
    node_list = sorted(nodes.items(), key=lambda x: x[1])
    node_names = [name for name, _ in node_list]
    
    link_sources, link_targets, link_values = zip(*links)
    
    sankey = go.Sankey(
        arrangement='snap',
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=node_names,
            color="blue",
            align='left'
        ),
        link=dict(
            source=link_sources,
            target=link_targets,
            value=link_values,
            arrowlen=15
        )
    )
    
    layout = go.Layout(
        title_text="Stablecoin Flow Sankey Diagram",
        font_size=10
    )
    
    fig = go.Figure(data=[sankey], layout=layout)
    return fig

sankey_fig = create_sankey(first_order, second_order)

sankey_fig.write_html("sankey_plot.html")
from IPython.display import IFrame
IFrame(src="sankey_plot.html", width=900, height=600)